In [2]:
import pandas as pd
import numpy as np
from sklearn import cross_validation, linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV

In [3]:
pwd

u'/home/ubuntu/repo/dsi/DOTA2_Recommendation-System/notebook'

In [4]:
cd ../

/home/ubuntu/repo/dsi/DOTA2_Recommendation-System


In [5]:
from data_pipeline.data_clean import read_file

In [5]:
filename = 'data_collection/match_history_sample.json'

In [7]:
# match_json_array = []
with open(filename) as f:
#     print sum(1 for _ in f)
   arr = [next(f) for x in xrange(300)]
#     for linze in f:
#         match_json_array.append(line)

In [8]:
t = map(lambda x: pd.read_json(x), arr)


In [9]:
df = pd.concat(t,ignore_index=True)

In [10]:
df = pd.DataFrame(df.matches.tolist())

In [11]:
df_10player = df[df['human_players']==10]

In [12]:
def match_features(players):
    vec = np.zeros(224, dtype=np.int)
    loc = np.append(np.zeros(5, dtype=np.int), np.ones(5, dtype=np.int)*112)
    loc+=map(lambda x: x['hero_id']-1, players)
    vec[loc]=1
    return vec

def matches_features(df):
    df_feature = df['players'].apply(lambda x: match_features(x))
    X = pd.DataFrame(df_feature.tolist())
    y = df['radiant_win']
    return X, y

In [13]:
X, y = matches_features(df_10player)

# Random Forest

In [14]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.8)

In [15]:
# Set the parameters by cross validation

tuned_parameters = [
                    {'n_estimators':[100, 1000, 10000]},
                    {'max_features':[5, 'auto', None]},
                    {'min_samples_leaf':[1, 50, 150]}
                    ]

In [16]:
clf_rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=50)

In [17]:
clf = GridSearchCV(clf_rf, tuned_parameters, cv=5)

In [18]:
clf.fit(X_train, y_train)

/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:403: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=True, random_state=50, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [100, 1000, 10000]}, {'max_features': [5, 'auto', None]}, {'min_samples_leaf': [1, 50, 150]}],
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [19]:
clf.best_params_                                            

{'n_estimators': 10000}

In [20]:
clf_rf =RandomForestClassifier(n_estimators=10000,n_jobs=-1, min_samples_leaf=50, random_state=50)

In [21]:
cross_validation.cross_val_score(clf_rf, X, y, cv=5)

array([ 0.58409365,  0.58013427,  0.58667585,  0.58288862,  0.58798209])

In [22]:
clf_log_r = linear_model.LogisticRegression()
cross_validation.cross_val_score(clf_log_r, X, y, cv=5)

array([ 0.60027543,  0.59149595,  0.59803753,  0.58977449,  0.5922865 ])